In [35]:
import pandas as pd
from typing import List
from pandas import DataFrame
from tabulate import tabulate
import os

In [36]:
def concat_path(folder_name, file_name):
  return os.path.join(folder_name, file_name)

In [37]:
def current_dir(dir_target: str) -> str:
  current_dir = os.getcwd()
  folder_path = concat_path(current_dir, dir_target)
  return folder_path

In [38]:
def get_csv_files(folder_name: str) -> List[str]:
  running_folder_path = current_dir(folder_name)
  files_in_running_folder = os.listdir(running_folder_path)
  files = sorted([concat_path(running_folder_path, file) for file in files_in_running_folder])

  return files

In [39]:
def edit_date_format(date: str):
  date_split = date.split(' ')
  del date_split[3]
  new_date_format = ' '.join(date_split)

  return new_date_format

In [40]:
def rename_column(df: DataFrame, old_name: str, new_name: str) -> None:
  df.rename(columns={old_name: new_name}, inplace=True)

In [41]:
def drop_column(df: DataFrame, column: str) -> None:
  df.drop(columns=[column], inplace=True)

In [42]:
def sec_to_min(sec: float) -> str:
  min = int(sec // 60)
  sec = int(sec % 60)

  if (len(str(sec)) == 1): sec = f'0{sec}'
  
  return f'{min}:{sec}'


sec_to_min(336.572727)


'5:36'

In [43]:
def convert_activity(activity):
  if (activity == 'Running (Indoor)'):
    return 'indoor'
  return 'outdoor'

In [44]:
def filter_interval(df: DataFrame):
  return df[(df['Distance(km)'] >= 1) & (df['Distance(km)'] <= 1.1)]

In [45]:
def filter_5K(df: DataFrame):
  return df[(df['Distance(km)'] >= 5) & (df['Distance(km)'] <= 5.1) ]

In [46]:
def filter_7K(df: DataFrame):
  return df[(df['Distance(km)'] >= 7) & (df['Distance(km)'] <= 7.1) ]

In [47]:
def filter_10K(df: DataFrame):
  return df[(df['Distance(km)'] >= 10) & (df['Distance(km)'] <= 10.1)]

In [48]:
def filter_longrun(df: DataFrame):
  return df[(df['Distance(km)'] >= 10.5)]

In [49]:
d = '2566-10-02 18:20:31 - 2566-10-02 18:39:08'
a = list(set(d.split(' ')))
k = [i for i in a if i != '-']

t = '-'.join(k[:2])
v = f'{t} {k[2]}'
v


'18:39:08-2566-10-02 18:20:31'

In [50]:
runningcsv_files = get_csv_files('running')

df = pd.concat( 
  map(pd.read_csv, runningcsv_files), 
  ignore_index=True
)

rename_column(df, 'Active energy burned(kcal)', 'Energy (kcal)')
rename_column(df, 'Heart rate: Average(count/min)', 'Heart rate: Average(min)')
rename_column(df, 'Heart rate: Maximum(count/min)', 'Heart rate: Maximum(min)')

drop_columns = [
  'Heart rate zone: A Easy (<115bpm)(%)',
  'Heart rate zone: B Fat Burn (115-135bpm)(%)',
  'Heart rate zone: C Moderate Training (135-155bpm)(%)',
  'Heart rate zone: D Hard Training (155-175bpm)(%)',
  'Heart rate zone: E Extreme Training (>175bpm)(%)',
  'Elevation: Ascended(m)',
  'Elevation: Maximum(m)',
  'Elevation: Minimum(m)',
  'METs Average(kcal/hr·kg)',
  'Weather: Humidity(%)',
  'Weather: Temperature(degC)',
]

for column in drop_columns:
  drop_column(df, column)

df['Pace(sec)'] = df['Duration(s)'] / df['Distance(km)']
df['Pace(min)'] = df['Pace(sec)'].apply(sec_to_min)
df['Activity'] = df['Activity'].apply(convert_activity)
df['Date'] = df['Date'].apply(edit_date_format)
df['Duration(min)'] = df['Duration(s)'].apply(sec_to_min)

column_order = ['Date', 'Energy (kcal)', 'Activity', 'Distance(km)', 'Duration(min)', 'Pace(min)', 'Heart rate: Average(min)', 'Heart rate: Maximum(min)']
d = df[column_order]

d



,Date,Energy (kcal),Activity,Distance(km),Duration(min),Pace(min),Heart rate: Average(min),Heart rate: Maximum(min)
0,2566-10-02 18:20:31 - 18:39:08,183.992,indoor,3.300,18:30,5:36,164.525,182.0
1,2566-10-03 18:34:49 - 19:03:43,267.510,indoor,4.631,28:14,6:06,166.159,197.0
2,2566-10-04 18:09:39 - 18:53:54,434.123,indoor,7.388,38:18,5:11,158.990,189.0
3,2566-10-05 10:06:21 - 10:34:35,306.220,indoor,5.565,27:54,5:00,160.902,183.0
4,2566-10-05 17:32:16 - 17:40:01,80.592,indoor,1.488,7:42,5:10,176.280,188.0
...,...,...,...,...,...,...,...,...
189,2567-07-23 18:18:36 - 18:42:58,136.704,outdoor,2.255,14:16,6:19,132.370,158.0
190,2567-07-24 17:49:21 - 18:21:45,261.219,outdoor,5.021,25:23,5:03,150.811,182.0
191,2567-07-24 18:36:27 - 18:57:01,272.044,outdoor,5.026,19:37,3:54,182.889,202.0
192,2567-07-26 17:15:38 - 18:04:14,339.880,outdoor,6.385,38:42,6:03,143.123,169.0


In [51]:
def filter_value(df: DataFrame, column: str, value: str) -> DataFrame:
  return df[df[column] == value]

In [52]:
outdoor = filter_value(df, 'Activity', 'outdoor')
outdoor.reset_index()[column_order]

,Date,Energy (kcal),Activity,Distance(km),Duration(min),Pace(min),Heart rate: Average(min),Heart rate: Maximum(min)
0,2566-10-07 07:57:08 - 08:17:11,179.592,outdoor,3.041,17:39,5:48,167.339,179.0
1,2566-10-14 17:30:23 - 17:58:10,253.719,outdoor,4.017,19:50,4:56,162.639,186.0
2,2566-10-21 17:57:09 - 18:16:47,147.069,outdoor,2.090,10:58,5:15,143.814,178.0
3,2566-10-23 08:02:10 - 08:31:26,267.240,outdoor,4.104,22:38,5:31,154.657,175.0
4,2566-10-30 17:41:32 - 18:15:11,291.367,outdoor,4.016,21:38,5:23,166.435,191.0
...,...,...,...,...,...,...,...,...
122,2567-07-23 18:18:36 - 18:42:58,136.704,outdoor,2.255,14:16,6:19,132.370,158.0
123,2567-07-24 17:49:21 - 18:21:45,261.219,outdoor,5.021,25:23,5:03,150.811,182.0
124,2567-07-24 18:36:27 - 18:57:01,272.044,outdoor,5.026,19:37,3:54,182.889,202.0
125,2567-07-26 17:15:38 - 18:04:14,339.880,outdoor,6.385,38:42,6:03,143.123,169.0


In [53]:
indoor = filter_value(df, 'Activity', 'indoor')
indoor.reset_index()[column_order]

,Date,Energy (kcal),Activity,Distance(km),Duration(min),Pace(min),Heart rate: Average(min),Heart rate: Maximum(min)
0,2566-10-02 18:20:31 - 18:39:08,183.992,indoor,3.300,18:30,5:36,164.525,182.0
1,2566-10-03 18:34:49 - 19:03:43,267.510,indoor,4.631,28:14,6:06,166.159,197.0
2,2566-10-04 18:09:39 - 18:53:54,434.123,indoor,7.388,38:18,5:11,158.990,189.0
3,2566-10-05 10:06:21 - 10:34:35,306.220,indoor,5.565,27:54,5:00,160.902,183.0
4,2566-10-05 17:32:16 - 17:40:01,80.592,indoor,1.488,7:42,5:10,176.280,188.0
...,...,...,...,...,...,...,...,...
62,2567-03-05 18:35:04 - 19:09:43,383.170,indoor,7.054,34:05,4:49,173.711,189.0
63,2567-03-06 18:17:35 - 18:50:43,384.148,indoor,7.043,33:08,4:42,169.310,178.0
64,2567-03-12 18:27:01 - 19:05:58,382.394,indoor,7.029,34:29,4:54,163.540,185.0
65,2567-03-13 18:19:25 - 18:57:36,378.653,indoor,7.019,35:06,5:00,164.913,184.0


In [54]:
filter_interval(df)

,Date,Energy (kcal),Activity,Distance(km),Duration(s),Heart rate: Average(min),Heart rate: Maximum(min),Pace(sec),Pace(min),Duration(min)
121,2567-05-16 18:54:11 - 19:01:29,56.798,outdoor,1.045,280.291,177.682,192.0,268.221053,4:28,4:40
127,2567-05-24 18:36:52 - 18:43:21,55.533,outdoor,1.031,266.386,176.352,191.0,258.376334,4:18,4:26
130,2567-05-27 18:45:58 - 19:03:53,56.124,outdoor,1.039,262.402,160.016,196.0,252.552454,4:12,4:22
133,2567-05-29 18:15:09 - 18:22:51,56.416,outdoor,1.036,250.446,173.478,189.0,241.743243,4:01,4:10
134,2567-05-29 18:23:19 - 18:31:41,55.689,outdoor,1.028,248.280,176.220,192.0,241.517510,4:01,4:08
135,2567-05-29 18:32:01 - 18:37:16,55.341,outdoor,1.042,257.321,184.492,195.0,246.949136,4:06,4:17
136,2567-05-29 18:41:59 - 18:53:53,54.066,outdoor,1.028,247.124,161.895,191.0,240.392996,4:00,4:07
137,2567-05-29 18:54:22 - 19:01:37,55.943,outdoor,1.024,240.559,171.747,190.0,234.920898,3:54,4:00
141,2567-06-03 18:42:59 - 18:49:03,55.494,outdoor,1.027,264.085,170.700,185.0,257.142162,4:17,4:24
143,2567-06-06 18:01:44 - 18:05:33,53.125,outdoor,1.041,229.247,183.361,190.0,220.218060,3:40,3:49


In [55]:
run5k = filter_5K(outdoor)
run5k

,Date,Energy (kcal),Activity,Distance(km),Duration(s),Heart rate: Average(min),Heart rate: Maximum(min),Pace(sec),Pace(min),Duration(min)
35,2566-11-13 17:34:55 - 18:06:49,273.554,outdoor,5.044,1858.161,163.325,176.0,368.390365,6:08,30:58
61,2566-12-28 07:37:58 - 08:07:19,266.105,outdoor,5.056,1464.858,142.500,161.0,289.726661,4:49,24:24
62,2566-12-28 17:44:22 - 18:11:36,266.066,outdoor,5.045,1477.712,168.188,178.0,292.906244,4:52,24:37
64,2567-01-02 07:28:04 - 07:54:03,267.671,outdoor,5.063,1487.952,164.318,174.0,293.887419,4:53,24:47
68,2567-01-06 17:45:50 - 18:12:15,268.547,outdoor,5.035,1359.984,181.058,201.0,270.106058,4:30,22:39
73,2567-01-13 17:49:44 - 18:13:43,268.425,outdoor,5.049,1438.501,175.733,184.0,284.908101,4:44,23:58
75,2567-01-16 17:57:52 - 18:22:36,267.875,outdoor,5.062,1388.165,166.141,179.0,274.232517,4:34,23:08
76,2567-01-17 17:53:35 - 18:16:36,272.726,outdoor,5.071,1380.688,175.953,188.0,272.271347,4:32,23:00
77,2567-01-18 17:55:43 - 18:50:13,268.990,outdoor,5.077,1338.707,140.719,193.0,263.680717,4:23,22:18
78,2567-01-19 17:48:30 - 18:11:47,271.672,outdoor,5.036,1338.811,182.986,191.0,265.848094,4:25,22:18


In [56]:
run7k = filter_7K(outdoor)
run7k

,Date,Energy (kcal),Activity,Distance(km),Duration(s),Heart rate: Average(min),Heart rate: Maximum(min),Pace(sec),Pace(min),Duration(min)
41,2566-11-22 17:17:00 - 18:06:44,361.793,outdoor,7.042,2549.349,161.180,174.0,362.020591,6:02,42:29
57,2566-12-22 17:40:06 - 18:13:32,374.418,outdoor,7.043,1966.304,180.819,187.0,279.185574,4:39,32:46
79,2567-01-20 17:53:21 - 18:26:55,374.721,outdoor,7.039,1987.794,172.833,184.0,282.397216,4:42,33:07
101,2567-02-25 17:51:05 - 18:23:58,377.729,outdoor,7.043,1917.996,186.358,198.0,272.326565,4:32,31:57
108,2567-03-09 17:59:04 - 18:31:08,375.394,outdoor,7.033,1886.370,175.895,191.0,268.216977,4:28,31:26
109,2567-03-10 17:57:45 - 18:29:11,377.391,outdoor,7.045,1875.851,181.804,192.0,266.266998,4:26,31:15
112,2567-03-15 18:15:23 - 18:48:50,376.427,outdoor,7.038,1959.521,178.816,186.0,278.420148,4:38,32:39
114,2567-05-03 18:21:18 - 19:01:59,373.897,outdoor,7.030,2343.601,172.104,194.0,333.371408,5:33,39:03
115,2567-05-05 18:20:32 - 19:15:29,377.770,outdoor,7.028,2156.188,164.770,194.0,306.799659,5:06,35:56
128,2567-05-25 18:23:31 - 19:14:44,371.488,outdoor,7.061,2072.546,156.806,193.0,293.520181,4:53,34:32


In [57]:
run10k = filter_10K(outdoor)
run10k

,Date,Energy (kcal),Activity,Distance(km),Duration(s),Heart rate: Average(min),Heart rate: Maximum(min),Pace(sec),Pace(min),Duration(min)
52,2566-12-16 17:26:05 - 18:16:58,563.603,outdoor,10.025,3028.179,181.844,192.0,302.062743,5:02,50:28
58,2566-12-23 17:24:30 - 18:13:01,529.661,outdoor,10.052,2848.219,182.775,194.0,283.348488,4:43,47:28
149,2567-06-07 17:56:23 - 19:08:14,539.981,outdoor,10.013,3513.891,161.056,179.0,350.932887,5:50,58:33
184,2567-07-17 17:47:12 - 18:56:46,532.442,outdoor,10.075,3125.050,153.784,198.0,310.178660,5:10,52:05


In [58]:
longrun = filter_longrun(outdoor)
longrun

,Date,Energy (kcal),Activity,Distance(km),Duration(s),Heart rate: Average(min),Heart rate: Maximum(min),Pace(sec),Pace(min),Duration(min)
152,2567-06-11 17:55:09 - 19:15:02,590.464,outdoor,10.632,3625.281,147.040,168.0,340.978273,5:40,60:25
159,2567-06-13 17:43:41 - 18:51:44,609.384,outdoor,10.551,3566.921,157.923,172.0,338.064733,5:38,59:26
162,2567-06-18 17:40:12 - 19:20:36,658.691,outdoor,12.017,4004.761,153.016,178.0,333.257968,5:33,66:44
169,2567-06-20 17:42:38 - 18:49:10,617.962,outdoor,11.029,3677.336,155.596,170.0,333.424245,5:33,61:17
181,2567-07-14 17:31:48 - 18:52:43,774.967,outdoor,13.250,4448.425,172.735,192.0,335.730189,5:35,74:08
186,2567-07-19 17:35:37 - 18:59:56,706.758,outdoor,13.010,4418.175,157.812,183.0,339.598386,5:39,73:38
193,2567-07-31 17:33:12 - 18:59:13,655.960,outdoor,11.832,4055.119,155.693,187.0,342.724730,5:42,67:35


In [59]:
def pace_table(data) -> str:
  fastest_pace = df.loc[data]

  table_data = {
    'Date': [fastest_pace['Date']],
    'Energy (kcal)': [fastest_pace['Energy (kcal)']],
    'Activity': [fastest_pace['Activity']],
    'Distance(km)': [fastest_pace['Distance(km)']],
    'Duration(s)': [fastest_pace['Duration(s)']],
    'Pace(min)': [fastest_pace['Pace(min)']],
    'Pace(sec)': [fastest_pace['Pace(sec)']],
    'Heart rate: Average(min)': [fastest_pace['Heart rate: Average(min)']],
    'Heart rate: Maximum(min)': [fastest_pace['Heart rate: Maximum(min)']]
  }

  return tabulate(table_data, headers='keys', tablefmt='pretty')

In [60]:
fastest_pace_outdoor = outdoor['Pace(sec)'].idxmin()
fastest_pace_indoor = indoor['Pace(sec)'].idxmin()

# fastest_outdoor_pace_table = pace_table(fastest_pace_outdoor)
# fastest_indoor_pace_table = pace_table(fastest_pace_indoor)

fastest_outdoor_pace_table = df.loc[fastest_pace_outdoor]
fastest_indoor_pace_table = df.loc[fastest_pace_indoor]

print('The fastest pace outdoor')
print(fastest_outdoor_pace_table)
print()

print('The fastest pace indoor')
print(fastest_indoor_pace_table)

The fastest pace outdoor
Date                        2567-02-17 18:24:30 - 18:28:02
Energy (kcal)                                       22.642
Activity                                           outdoor
Distance(km)                                         0.399
Duration(s)                                         76.541
Heart rate: Average(min)                            166.52
Heart rate: Maximum(min)                             194.0
Pace(sec)                                        191.83208
Pace(min)                                             3:11
Duration(min)                                         1:16
Name: 97, dtype: object

The fastest pace indoor
Date                        2567-01-30 17:33:17 - 17:56:55
Energy (kcal)                                      286.131
Activity                                            indoor
Distance(km)                                          5.26
Duration(s)                                       1415.417
Heart rate: Average(min)                 

In [61]:
def date_selector(df: DataFrame, date: str) -> DataFrame:
  return df[df['Date'].astype(str).str.contains(date)]

In [62]:
def mean_column(df: DataFrame, column: str):
  return round(df[column].mean(), 2)

In [63]:
distance = df['Distance(km)'].sum()
print(f'Overall {distance:.2f} km')

Overall 924.30 km


In [64]:
fastest5k = run5k['Pace(min)'].idxmin()
print("Fatest of 5 KM.")
run5k.loc[fastest5k]

Fatest of 5 KM.


Date                        2567-07-24 18:36:27 - 18:57:01
Energy (kcal)                                      272.044
Activity                                           outdoor
Distance(km)                                         5.026
Duration(s)                                       1177.557
Heart rate: Average(min)                           182.889
Heart rate: Maximum(min)                             202.0
Pace(sec)                                       234.293076
Pace(min)                                             3:54
Duration(min)                                        19:37
Name: 191, dtype: object

In [65]:
fastest7k = run7k['Pace(min)'].idxmin()
print("Fatest of 7 KM.")
run7k.loc[fastest7k]

Fatest of 7 KM.


Date                        2567-06-08 18:13:40 - 19:01:30
Energy (kcal)                                      370.562
Activity                                           outdoor
Distance(km)                                         7.044
Duration(s)                                       1852.672
Heart rate: Average(min)                           152.381
Heart rate: Maximum(min)                             190.0
Pace(sec)                                       263.014196
Pace(min)                                             4:23
Duration(min)                                        30:52
Name: 150, dtype: object

In [66]:
fastest10k = run10k['Pace(min)'].idxmin()
print("Fatest of 10 KM.")
run10k.loc[fastest10k]

Fatest of 10 KM.


Date                        2566-12-23 17:24:30 - 18:13:01
Energy (kcal)                                      529.661
Activity                                           outdoor
Distance(km)                                        10.052
Duration(s)                                       2848.219
Heart rate: Average(min)                           182.775
Heart rate: Maximum(min)                             194.0
Pace(sec)                                       283.348488
Pace(min)                                             4:43
Duration(min)                                        47:28
Name: 58, dtype: object

In [67]:
date = '2567-07'
print('Data on', date)

distance = 'Distance(km)'
indoor = 'indoor'
pace = 'Pace(sec)'
outdoor = 'outdoor'

running = date_selector(df, date)

def mean(column: str):
  return mean_column(running, column)

def find_best_data_column(activity: str, column: str):
  activity_condition = (running['Activity'] == activity)
  distance_condition = (running[distance] > 5)
  condition = activity_condition & distance_condition
  filtered_df = running[condition][column]
  try:
    if column == pace:
      return filtered_df.idxmin()
    
    return filtered_df.idxmax()
  except: return None

def best_loc_data(idx, column: str) -> DataFrame:
  if idx is None:
    return None
  if column == pace:
    return sec_to_min(running.loc[idx][column])
  
  return running.loc[idx][column]

avg_distance = mean(distance)
avg_duration = sec_to_min(mean('Duration(s)'))
avg_pace = sec_to_min(mean(pace))
avg_heart_rate = mean('Heart rate: Average(min)')
avg_max_heart_rate = mean('Heart rate: Maximum(min)')

id_fastest_pace_indoor = find_best_data_column(indoor, pace)
fastest_pace_indoor = best_loc_data(id_fastest_pace_indoor, pace)
id_fastest_pace_outdoor = find_best_data_column(outdoor, pace)
fastest_pace_outdoor = best_loc_data(id_fastest_pace_outdoor, pace)


id_longest_distance_indoor = find_best_data_column(indoor, distance)
longest_distance_indoor = best_loc_data(id_longest_distance_indoor, distance)
id_longest_distance_outdoor = find_best_data_column(outdoor, distance)
longest_distance_outdoor = best_loc_data(id_longest_distance_outdoor, distance)

all_distance = running['Distance(km)'].sum()


print(len(running), 'runs')
print(f'{all_distance:.2f} km')
print()
print('Fastest run indoor', fastest_pace_indoor, 'km')
print('Fastest run outdoor', fastest_pace_outdoor, 'km')
print()
print('Longest distance indoor', longest_distance_indoor, 'km')
print('Longest distance outdoor', longest_distance_outdoor, 'km')
print()
print('Average distance', avg_distance, 'km')
print('Average time', avg_duration, 'min')
print('Average pace', avg_pace, 'min')
print('Average heart rate', avg_heart_rate, 'per min')
print('Average max heart rate', avg_max_heart_rate, 'per min')

Data on 2567-07
16 runs
110.27 km

Fastest run indoor None km
Fastest run outdoor 3:54 km

Longest distance indoor None km
Longest distance outdoor 13.25 km

Average distance 6.89 km
Average time 38:16 min
Average pace 5:38 min
Average heart rate 147.79 per min
Average max heart rate 175.06 per min
